In [ ]:
#1. Sorting Windows Executable Files 📊

import os
import shutil
import subprocess

def is_windows_executable(filepath):
    try:
        output = subprocess.check_output(["file", "--brief", filepath]).decode("utf-8").strip()
        return "PE32" in output or "PE32+" in output
    except subprocess.CalledProcessError:
        return False

def copy_windows_executables(source_folder, destination_folder):
    try:
        os.makedirs(destination_folder, exist_ok=True)

        for root, _, files in os.walk(source_folder):
            for filename in files:
                file_path = os.path.join(root, filename)
                if is_windows_executable(file_path):
                    destination_path = os.path.join(destination_folder, filename)
                    shutil.move(file_path, destination_path)
                    print(f"Moved '{filename}' to '{destination_folder}'.")
                else:
                    print(f"Deleting '{filename}' permanently.")
                    os.remove(file_path)

        print("File check and copying completed.")
    except Exception as e:
        print("An error occurred:", e)

if __name__ == "__main__":
    source_folder = ""              # absolute path to mixedMalwares
    destination_folder = ""         # absolute path to onlyPrograms

    copy_windows_executables(source_folder, destination_folder)


In [ ]:
#2 Renaming to MD5 Hash Name ✏️
# For VirusShare torrents only.

import os

def rename_files(folder):
    try:
        for root, _, files in os.walk(folder):
            for filename in files:
                if filename.startswith("VirusShare_"):
                    old_path = os.path.join(root, filename)
                    new_filename = filename.replace("VirusShare_", "")
                    new_path = os.path.join(root, new_filename)
                    os.rename(old_path, new_path)

        print("File renaming completed.")
    except Exception as e:
        print("An error occurred:", e)
        
        
if __name__ == "__main__":
    
    destination_folder = ""          # absolute path to onlyPrograms
    
    rename_files(destination_folder)

In [ ]:
#2 Renaming to MD5 Hash Name ✏️
# For Malware Baazar files only.
# Nedded as the onlyPrograms folders sometimes have .dll files also.

import os
import hashlib

folder_path = ""        # absolute path to onlyPrograms

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    if filename.endswith(".exe") and os.path.isfile(file_path):
        md5_hash = hashlib.md5()
        with open(file_path, "rb") as file:
            while chunk := file.read(8192):
                md5_hash.update(chunk)
        
        md5_hex = md5_hash.hexdigest()
        
        new_filename = md5_hex
        
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(file_path, new_file_path)
        print(f"Renamed '{filename}' to '{new_filename}'")
    
    elif not filename.endswith(".exe") and os.path.isfile(file_path):
        os.remove(file_path)
        print(f"Deleted '{filename}'")

In [ ]:
#3 Getting JSON report of each file using VirusTotal 📄

import os
import requests
import json
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timezone, timedelta

VIRUSTOTAL_API_KEYS = ["", ""]

def load_processed_hashes():
    processed_hashes = set()
    if os.path.exists("processed_hashes.txt"):
        with open("processed_hashes.txt", "r") as f:
            processed_hashes = set(line.strip() for line in f.readlines())
    return processed_hashes

def save_processed_hash(hash_name):
    with open("processed_hashes.txt", "a") as f:
        f.write(hash_name + "\n")

def get_virustotal_report(hash_name, api_key):
    url = f"https://www.virustotal.com/api/v3/files/{hash_name}"
    headers = {
        "x-apikey": api_key,
        "accept": "application/json"
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        save_processed_hash(hash_name)
        return response.json()
    elif response.status_code == 429:
        print(f"API quota for API key {api_key} is exhausted.")
        wait_time = calculate_wait_time()
        print(f"The quota will be restored in {wait_time} minutes.")
        wait_until_quota_restored(wait_time)
        return get_virustotal_report(hash_name, api_key)
    else:
        print(f"Error occurred for {hash_name}: {response.status_code} - {response.text}")
        return None

def save_json_report(hash_name, json_report):
    folder_path = ""                # absolute path to json_reports
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    file_path = os.path.join(folder_path, f"{hash_name}.json")
    with open(file_path, "w") as f:
        json.dump(json_report, f, indent=2)

def process_files(api_key, files):
    for filename in files:
        hash_name = os.path.splitext(filename)[0]

        json_report = get_virustotal_report(hash_name, api_key)

        if json_report is not None:
            save_json_report(hash_name, json_report)
            print(f"Saved JSON report for {filename}")

def calculate_wait_time():
    now = datetime.now(timezone.utc)
    daily_reset_time = now.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1)
    time_remaining = daily_reset_time - now
    wait_time = time_remaining.total_seconds() // 60
    return wait_time

def wait_until_quota_restored(wait_time):
    current_time = datetime.now(timezone.utc)
    daily_reset_time = current_time.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1)
    target_time = current_time + timedelta(minutes=wait_time)
    while current_time < daily_reset_time:
        remaining_time = target_time - current_time
        print(f"The quota will be restored in {int(remaining_time.total_seconds() // 60)} minutes.")
        time.sleep(300)
        current_time = datetime.now(timezone.utc)
    print("API quota restored. Reporting will resume in 1 minute.")
    time.sleep(60)

def main():
    folder_path = ""                # absolute path to onlyPrograms
    files = os.listdir(folder_path)
    num_files = len(files)
    mid = num_files // 2

    processed_hashes = load_processed_hashes()
    files_for_api1 = [filename for filename in files[:mid] if os.path.splitext(filename)[0] not in processed_hashes]
    files_for_api2 = [filename for filename in files[mid:] if os.path.splitext(filename)[0] not in processed_hashes]

    with ThreadPoolExecutor(max_workers=2) as executor:
        future1 = executor.submit(process_files, VIRUSTOTAL_API_KEYS[0], files_for_api1)
        future2 = executor.submit(process_files, VIRUSTOTAL_API_KEYS[1], files_for_api2)
        results = [future.result() for future in as_completed([future1, future2])]

if __name__ == "__main__":
    main()

In [ ]:
#4 Verify from reports and remove any non-executable file 🔍✔️
# Should be performed to verify that the files in onlyProgram are really windows executables or not.

import os
import json
import shutil

def is_win32_exe(json_data):
    file_type = json_data['data']['attributes'].get('type_description', 'No Data')
    return file_type == 'Win32 EXE'

def process_json_reports(report_folder, main_folder):
    for json_file in os.listdir(report_folder):
        if json_file.endswith('.json'):
            json_file_path = os.path.join(report_folder, json_file)
            main_file_path = os.path.join(main_folder, json_file[:-5])

            with open(json_file_path, 'r') as f:
                json_data = json.load(f)

            if is_win32_exe(json_data):
                print(f"{json_file} is Win32 EXE. Keeping both JSON report and main file.")
            else:
                print(f"{json_file} is not Win32 EXE. Deleting JSON report and main file.")
                os.remove(json_file_path)
                os.remove(main_file_path)

def main():
    reports_folder = ""         # absolute path to json_reports
    main_folder = ""            # absolute path to onlyPrograms

    process_json_reports(reports_folder, main_folder)

if __name__ == "__main__":
    main()


In [ ]:
#5 Remove any executable with no report found 🗑️

import os

def read_txt_file(file_path):
    with open(file_path, 'r') as file:
        return set(file.read().splitlines())

def delete_unreported_files(executables_folder, json_reports_folder):
    deleted_count = 0
    for root, _, files in os.walk(executables_folder):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            
            hash_name = os.path.splitext(file_name)[0]
            
            json_report_file = os.path.join(json_reports_folder, f"{hash_name}.json")
            if not os.path.exists(json_report_file):
                try:
                    os.remove(file_path)
                    deleted_count += 1
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Error deleting {file_path}: {str(e)}")
    
    return deleted_count

if __name__ == "__main__":
    executables_folder = ""             # absolute path to onlyPrograms
    json_reports_folder = ""            # absolute path to json_reports

    deleted_count = delete_unreported_files(executables_folder, json_reports_folder)

    print(f"Total unreported files deleted: {deleted_count}")

In [ ]:
#6 Move the latest executables to a different folder. 🚚

import os
import json
import shutil
from datetime import datetime

only_programs_folder = ''           # absolute path to onlyPrograms
json_reports_folder = ''            # absolute path to json_reports
new_json_folder = ''                # absolute path to new_malware_json
new_mal_folder = ''                 # absolute path to new_malware


if not os.path.exists(new_json_folder):
    os.makedirs(new_json_folder)

if not os.path.exists(new_mal_folder):
    os.makedirs(new_mal_folder)

for json_file in os.listdir(json_reports_folder):
    json_path = os.path.join(json_reports_folder, json_file)

    if not json_file.endswith('.json') or not os.path.isfile(json_path):
        continue

    with open(json_path, 'r') as f:
        json_data = json.load(f)
    
    creation_date = json_data['data']['attributes'].get('creation_date', None)

    if creation_date is not None:
        unix_timestamp = int(creation_date)
        submission_date = datetime.utcfromtimestamp(unix_timestamp).strftime('%Y-%m-%d')

        if 2016 <= int(submission_date[:4]) <= 2023:
            new_json_path = os.path.join(new_json_folder, json_file)
            shutil.move(json_path, new_json_path)

            md5_hash = json_file.split('.')[0]
            executable_path = os.path.join(only_programs_folder, md5_hash)
            new_mal_path = os.path.join(new_mal_folder, md5_hash)
            shutil.move(executable_path, new_mal_path)

            print(f"Moved {json_file} and corresponding executable to new folders.")

In [ ]:
#7 Check for any empty JSON report 📄✔️

import os
import json

folder_path = ''       # absolute path to new_malware_json

if not os.path.exists(folder_path):
    print(f"Folder not found: {folder_path}")
else:
    empty_json_files = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)
            try:
                with open(file_path, 'r') as file:
                    data = json.load(file)
                    if not data:
                        empty_json_files.append(filename)
                        os.remove(file_path)
                        print(f"Deleted: {filename}")
            except (ValueError, FileNotFoundError):
                pass

    if empty_json_files:
        print(f"Total empty JSON files deleted: {len(empty_json_files)}")
    else:
        print("No empty JSON files found in the folder.")

In [ ]:
#8 Check for any JSON report with incorrect format 🔠

import os
import json

folder_path = ''  # absolute path to new_malware_json

if not os.path.exists(folder_path):
    print(f"Folder not found: {folder_path}")
else:
    invalid_json_files = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)
            try:
                with open(file_path, 'r') as json_file:
                    json_data = json.load(json_file)
            except json.JSONDecodeError:
                invalid_json_files.append(filename)
                os.remove(file_path)
                print(f"Deleted: {filename}")
            except FileNotFoundError:
                pass

    if invalid_json_files:
        print(f"Total invalid JSON files deleted: {len(invalid_json_files)}")
    else:
        print("No files with invalid JSON format found in the folder.")

In [ ]:
#9 Main classifier program 🗂️⚙️          TESTING... RUNNING PROPERLY;

import os
import re
import json
import subprocess
import shutil
import time
from datetime import datetime

def extract_highest_confidence_label(avclass_output, label_prefix):
    labels = re.findall(rf'{label_prefix}:(\w+\|\d+)', avclass_output)
    if labels:
        max_label = max(labels, key=lambda x: int(x.split("|")[-1]))
        return max_label.split("|")[0]
    else:
        return "unknown"

def run_avclass(avclass_input_path):
    avclass_command = f'avclass -f {avclass_input_path} -t'
    avclass_output = subprocess.check_output(avclass_command, shell=True, text=True)
    return avclass_output

def convert_to_one_liner(input_path, output_path):
    with open(input_path, 'r') as json_file:
        data = json.load(json_file)
    with open(output_path, 'w') as json_file:
        json.dump(data, json_file, separators=(',', ':'))

def organize_file(class_label, family_label, executable_md5, creation_date_y):
    base_folder = ""                                  # absolute path to your dataset root folder. Inside this it will start making the class folder and insode those their family folder and so on.
    class_folder = os.path.join(base_folder, class_label)
    family_folder = os.path.join(class_folder, family_label)
    
    unix_timestamp = int(creation_date_y)
    creation_date = datetime.utcfromtimestamp(unix_timestamp).strftime('%Y')

    if not os.path.exists(base_folder):
        os.makedirs(base_folder)

    if not os.path.exists(class_folder):
        os.makedirs(class_folder)

    if not os.path.exists(family_folder):
        os.makedirs(family_folder)

    file_counter = 1
    
   
    formatted_filename = f'{family_label}_{creation_date}_{file_counter}.exe'

    new_executable_name = os.path.join(family_folder, formatted_filename)

    while os.path.exists(new_executable_name):
        file_counter += 1
        formatted_filename = f'{class_label}_{family_label}_{creation_date}_{file_counter}.exe'
        new_executable_name = os.path.join(family_folder, formatted_filename)

    shutil.move(executable_md5, new_executable_name)


def process_reports():
    executable_folder = ""                          # absolute path to new_malware
    report_folder = ""                              # absolute path to new_malware_json
    avclassinput_path = "avclassinput.json"         # absolute path to avclassinput.json. This file is created when we are changing the json to oneliner.
    non_class_folder = "nonClassMalwares"           # absolute path to nonClassMalwares folder

    if not os.path.exists(non_class_folder):
        os.makedirs(non_class_folder)

    family_folders = {} 

    for report_file in os.listdir(report_folder):
        if report_file.endswith(".json"):
            json_path = os.path.join(report_folder, report_file)
            md5_hash = os.path.splitext(report_file)[0]
            executable_path = os.path.join(executable_folder, md5_hash)
            
            with open(json_path, 'r') as json_file:
                json_data = json.load(json_file)

            creation_date_x = json_data['data']['attributes'].get('creation_date', None)


            convert_to_one_liner(json_path, avclassinput_path)

            avclass_output = run_avclass(avclassinput_path)

            class_label = extract_highest_confidence_label(avclass_output, 'CLASS')
            family_label = extract_highest_confidence_label(avclass_output, 'FAM')

            if class_label != "unknown":
                organize_file(class_label, family_label, executable_path, creation_date_x)
            elif family_label != "unknown":
                if family_label not in family_folders:
                    family_folder = os.path.join(non_class_folder, family_label)
                    if not os.path.exists(family_folder):
                        os.makedirs(family_folder)
                    family_folders[family_label] = family_folder

                creation_date_a = json_data['data']['attributes'].get('creation_date', None)
                unix_timestamp = int(creation_date_a)
                creation_date_b = datetime.utcfromtimestamp(unix_timestamp).strftime('%Y%m')

                file_counter = 1

                while True:
                    formatted_filename = f'{family_label}_{creation_date_b}_{file_counter}.exe'
                    new_executable_name = os.path.join(family_folders[family_label], formatted_filename)

                    if not os.path.exists(new_executable_name):
                        break

                    file_counter += 1

                shutil.move(executable_path, new_executable_name)
            else:
                print(f"No valid class or family labels found in {report_file}")

if __name__ == '__main__':
    process_reports()

In [ ]:
#10 Delete orphaned JSON reports 📄🗑️
# Only when the categorization was stopped mid process and the executables have moved but json files are still tere in the folder.

# import os
# import json

# json_folder = 'new_malware_json'
# executable_folder = 'new_malware'

# json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

# for json_file in json_files:
    
#     base_filename = os.path.splitext(json_file)[0]
    
    
#     executable_path = os.path.join(executable_folder, base_filename)
    
    
#     if not os.path.exists(executable_path):
        
#         json_file_path = os.path.join(json_folder, json_file)
#         os.remove(json_file_path)
#         print(f"Deleted {json_file}")

# print("Cleanup complete.")

In [ ]:
#11 Final checks ✔️
# Correct files names. Adds the missing class name from the files.

import os

def fix_file_names(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.count('_') < 3:
                malware_class = os.path.basename(os.path.dirname(root))
                file_name, file_ext = os.path.splitext(file)
                new_file_name = f"{malware_class}_{file_name}{file_ext}"
                new_file_path = os.path.join(root, new_file_name)
                if os.path.exists(new_file_path):
                    version = 1
                    while True:
                        new_file_path = os.path.join(root, f"{malware_class}_{file_name}_{version}{file_ext}")
                        if not os.path.exists(new_file_path):
                            break
                        version += 1
                print(f"Renaming '{file}' to '{new_file_path}'")
                os.rename(os.path.join(root, file), new_file_path) 

folder_path = ''                    # absolute path to your dataset root folder.
fix_file_names(folder_path)

In [ ]:
#12 Class and Family count CSV 🗒️
# Store the malware class and family data in csv form.

import os
import csv

malware_folder = ''                 # absolute path to your dataset root folder.

class_folders = [folder for folder in os.listdir(malware_folder) if os.path.isdir(os.path.join(malware_folder, folder))]

unique_family_names = set()

class_family_names = {}

class_family_file_counts = {}

for class_folder in class_folders:
    class_folder_path = os.path.join(malware_folder, class_folder)

    family_folders = [folder for folder in os.listdir(class_folder_path) if os.path.isdir(os.path.join(class_folder_path, folder))]

    unique_family_names.update(family_folders)

    class_family_names[class_folder] = family_folders

    class_family_file_counts[class_folder] = {family: 0 for family in family_folders}

    for family_name in family_folders:
        family_folder_path = os.path.join(class_folder_path, family_name)
        file_count = sum(1 for file in os.listdir(family_folder_path) if file.endswith('.exe'))
        class_family_file_counts[class_folder][family_name] = file_count

sorted_family_names = sorted(list(unique_family_names))

with open('malware_categories.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    csvwriter.writerow(['Class/Family'] + sorted_family_names)

    for class_folder in class_folders:
        row_data = [class_folder]

        for family_name in sorted_family_names:
            file_count = class_family_file_counts[class_folder].get(family_name, 0)
            row_data.append(file_count)

        csvwriter.writerow(row_data)

    total_row = ["Total"]

    for family_name in sorted_family_names:
        total_count = sum(class_family_file_counts[class_folder].get(family_name, 0) for class_folder in class_folders)
        total_row.append(total_count)

    csvwriter.writerow(total_row)

In [ ]:
#12.1 Class-Year count CSV 🗒️
# Store malware families in a year-wise format in a csv.

import os
import csv

malware_folder = ''                 # absolute path to your dataset root folder.

family_year_counts = {}

for class_folder in os.listdir(malware_folder):
    class_folder_path = os.path.join(malware_folder, class_folder)

    if os.path.isdir(class_folder_path):
        for family_folder in os.listdir(class_folder_path):
            family_folder_path = os.path.join(class_folder_path, family_folder)

            if os.path.isdir(family_folder_path):
                if family_folder not in family_year_counts:
                    family_year_counts[family_folder] = {str(year): 0 for year in range(2016, 2024)}

                for file_name in os.listdir(family_folder_path):
                    if file_name.endswith('.exe'):
                        parts = file_name.split('_')
                        if len(parts) >= 4 and len(parts[2]) == 4:
                            year = parts[2] 
                            if year.isdigit() and int(year) in range(2016, 2024):
                                family_year_counts[family_folder][year] += 1
                            else:
                                print(f"Unexpected year format in file: {file_name}")
                        else:
                            print(f"Unexpected file name format: {file_name}")

sorted_family_names = sorted(family_year_counts.keys())

with open('family_year_counts.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    csvwriter.writerow(['Family'] + [str(year) for year in range(2016, 2024)] + ['Total'])

    for family_name in sorted_family_names:
        counts = [family_year_counts[family_name].get(str(year), 0) for year in range(2016, 2024)]
        total_count = sum(counts)
        csvwriter.writerow([family_name] + counts + [total_count])

    total_row = ["Total"]
    for year in range(2016, 2024):
        total_year_count = sum(family_year_counts[family_name].get(str(year), 0) for family_name in sorted_family_names)
        total_row.append(total_year_count)
    csvwriter.writerow(total_row)

In [ ]:
#12.2 Family-Year count CSV 🗒️
# Store all malware class in a year-wise format in a csv.

import os
import csv

malware_folder = ''                     # absolute path to your dataset root folder.

class_year_counts = {}

for class_folder in os.listdir(malware_folder):
    class_folder_path = os.path.join(malware_folder, class_folder)

    if os.path.isdir(class_folder_path):
        if class_folder not in class_year_counts:
            class_year_counts[class_folder] = {str(year): 0 for year in range(2016, 2024)}

        for family_folder in os.listdir(class_folder_path):
            family_folder_path = os.path.join(class_folder_path, family_folder)

            if os.path.isdir(family_folder_path):
                for file_name in os.listdir(family_folder_path):
                    if file_name.endswith('.exe'):
                        parts = file_name.split('_')
                        if len(parts) >= 4 and len(parts[2]) == 4:
                            year = parts[2]
                            if year.isdigit() and int(year) in range(2016, 2024):
                                class_year_counts[class_folder][year] += 1
                            else:
                                print(f"Unexpected year format in file: {file_name}")
                        else:
                            print(f"Unexpected file name format: {file_name}")

sorted_class_names = sorted(class_year_counts.keys())

with open('class_year_counts.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    csvwriter.writerow(['Class'] + [str(year) for year in range(2016, 2024)] + ['Total'])

    for class_name in sorted_class_names:
        counts = [class_year_counts[class_name].get(str(year), 0) for year in range(2016, 2024)]
        total_count = sum(counts)
        csvwriter.writerow([class_name] + counts + [total_count])

    total_row = ["Total"]
    for year in range(2016, 2024):
        total_year_count = sum(class_year_counts[class_name].get(str(year), 0) for class_name in sorted_class_names)
        total_row.append(total_year_count)
    csvwriter.writerow(total_row)